In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import RegexpTokenizer as regextoken
from nltk.corpus import stopwords
from nltk import FreqDist, bigrams, trigrams,ngrams
from nltk import WordNetLemmatizer
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import collections
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
# import gensim
# from gensim.models.keyedvectors import KeyedVectors
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D, Dropout, LSTM, GRU
from keras.regularizers import l1, l2
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
zomato = pd.read_csv("zomato.csv", na_values = ["-", ""])
# Making a copy of the data to work on
data = zomato.copy()
data.drop(['url','address','menu_item','phone','location'],axis=1,inplace=True)
data.rename(columns={'listed_in(city)':'location'}, inplace=True)
data=data[data['rate'].notnull()]

Using TensorFlow backend.


In [ ]:
# ch=data[data['name'].str.contains('urch Street Social')].head(1).copy()
# ch=ch.drop('listed_in(type)',axis=1)
# ch

In [2]:
sam=data[['reviews_list','location']].groupby(['location'],as_index=False).agg(lambda x :' '.join(x))

In [3]:
heybi={}
for i in range(0,30):
    s=sam[sam.index==i]
    s["reviews_list"]= s["reviews_list"].apply(lambda x: x.lower())
    tokenizer = regextoken("[a-zA-Z]+") 
    review_tokens =s["reviews_list"].apply(tokenizer.tokenize)
    stop = stopwords.words("english")
    stop.extend(["rated", "n", "nan", "x"])
    review_tokens = review_tokens.apply(lambda x: [token for token in x if token not in stop])
    all_reviews = review_tokens.astype(str).str.cat()
    cleaned_reviews = tokenizer.tokenize(all_reviews)
    j=s['location'].values[0]
    heybi[j] = cleaned_reviews

In [4]:
three={}
for i in heybi.keys():
    trigram = lambda a: zip(a, a[1:],a[2:])
    ap=list(trigram(heybi[i]))
    trigramFreq = collections.Counter(ap)
    three[i]=trigramFreq.most_common(10)  

In [5]:
print(three['BTM'])

[(('must', 'visit', 'place'), 721), (('veg', 'non', 'veg'), 706), (('place', 'hangout', 'friends'), 667), (('place', 'hang', 'friends'), 596), (('north', 'indian', 'food'), 498), (('good', 'food', 'good'), 458), (('food', 'really', 'good'), 455), (('nfood', 'nambience', 'nservice'), 396), (('good', 'place', 'hangout'), 376), (('nfood', 'nservice', 'nambience'), 353)]


In [6]:
print(three['Banashankari'])

[(('south', 'indian', 'food'), 129), (('must', 'visit', 'place'), 97), (('good', 'food', 'good'), 89), (('paneer', 'tikka', 'biriyani'), 86), (('north', 'indian', 'chinese'), 83), (('north', 'indian', 'food'), 79), (('worst', 'food', 'ever'), 70), (('paneer', 'butter', 'masala'), 68), (('delious', 'good', 'need'), 58), (('good', 'need', 'sum'), 58)]


In [8]:
print(three['Church Street'])

[(('must', 'visit', 'place'), 425), (('peri', 'peri', 'chicken'), 390), (('veg', 'non', 'veg'), 382), (('place', 'hangout', 'friends'), 360), (('food', 'really', 'good'), 319), (('one', 'best', 'places'), 318), (('nfood', 'nambience', 'nservice'), 272), (('mg', 'road', 'metro'), 270), (('overall', 'good', 'experience'), 238), (('road', 'metro', 'station'), 233)]


In [9]:
print(three['MG Road'])

[(('f', 'f', 'f'), 2353), (('f', 'c', 'f'), 1928), (('c', 'f', 'f'), 1377), (('f', 'f', 'c'), 1355), (('must', 'visit', 'place'), 752), (('veg', 'non', 'veg'), 606), (('peri', 'peri', 'chicken'), 560), (('one', 'best', 'places'), 455), (('place', 'hangout', 'friends'), 445), (('nfood', 'nambience', 'nservice'), 415)]


In [10]:
print(three['Residency Road'])

[(('f', 'f', 'f'), 1798), (('f', 'c', 'f'), 1158), (('c', 'f', 'f'), 778), (('f', 'f', 'c'), 740), (('must', 'visit', 'place'), 623), (('veg', 'non', 'veg'), 482), (('one', 'best', 'places'), 421), (('xa', 'xa', 'xa'), 377), (('peri', 'peri', 'chicken'), 376), (('place', 'hangout', 'friends'), 365)]


In [7]:
print(three['Old Airport Road'])

[(('f', 'f', 'f'), 1850), (('f', 'c', 'f'), 585), (('xa', 'xa', 'xa'), 453), (('north', 'indian', 'food'), 412), (('must', 'visit', 'place'), 400), (('c', 'f', 'f'), 374), (('veg', 'non', 'veg'), 349), (('f', 'e', 'f'), 306), (('e', 'f', 'e'), 278), (('f', 'f', 'c'), 269)]


In [ ]:
three.keys()

In [ ]:
# voting=data[['name','location','votes']].groupby(['location']).sum()

In [ ]:
# rt=data[['name','location','rate','votes','reviews_list']][(data['location']=='Banashankari') &(data['rate']!='NEW')].sort_values(by='rate').copy()

In [ ]:
# rt['rate']=rt['rate'].str[:3]
# rt['rate']=rt['rate'].astype(float)

In [ ]:
# data['rate']=data['rate'].str[:3]
# rt['rate']=rt['rate'].astype(float)
# rt[(rt['rate']>=4.0)& (rt['votes']<200)].sort_values(by=['rate','votes'])

# ======================

In [ ]:
# bad=rt[rt['name']=='Udupi Ruchi Grand'].head(1).iloc[:,:-1]
# bad

## [('Rated 1.0', 'RATED\n Ordered on zomato delivery for curiosity... Food was good but the quantity was as good as nothing... The 2 portions fo bhel puri received was not even the half you normally get from other places... Half of the silver container was pressed from bottom to appear the contents to be full... Pav bhaji was received with 3/4 of the plastic container without chopped onions and lemon.... Waste of money...')

## 'Rated 3.0', 'RATED\n The newly opened one at a prime location offers south Indian and north Indian cuisines.. I had ordered paneer tikka masala on a rainy day, so cheers to the service .. I received the order on time. The food dint taste great but not bad to be tried too , a good one flavoursome gravy..')

# ======================

In [ ]:
# voting.sort_values(by='votes',ascending=False).tail()
#28th lowest no.of votes

In [ ]:
# ptr=data[['name','location','rate','votes','approx_cost(for two people)']][(data['location']=='Old Airport Road') &(data['rate']!='NEW')].sort_values(by='rate').copy()

In [ ]:
# ptr['rate']=ptr['rate'].str[:3]
# ptr['rate']=ptr['rate'].astype(float)

In [ ]:
# ptr['approx_cost(for two people)']=ptr['approx_cost(for two people)'].str.replace(',','')

# ptr['approx_cost(for two people)']=ptr['approx_cost(for two people)'].astype(int)

In [ ]:
# ptr=ptr[ptr['approx_cost(for two people)'].notnull()]

In [ ]:
# ptr[(ptr['rate']>4) & (ptr['approx_cost(for two people)']>1000)].sort_values(by='approx_cost(for two people)',ascending=False)

In [ ]:
# ptr[(ptr['rate']>4) & (ptr['approx_cost(for two people)']>1000)]